In [1]:
import csv
import time
import pandas as pd
import random
import utils
import os
import multiprocessing
import redis
import json
from functools import partial
from classes.gift import Gift
from classes.trip import Trip

In [2]:
DATA_PATH = "./data/gifts.csv"
SLEIGH_CAPACITY = 1000
INITIAL_POPULATION_SIZE = 200
MAX_ITERATIONS = 100000
TOP_K = 10

In [3]:
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)

## Read data into list

In [ ]:
start = time.time()
print("Initiating Dataset Loading...")

df = pd.read_csv(DATA_PATH)
gift_list = []

for i in range(len(df)):
    gift_series = df.iloc[i]
    gift = {
        'gift_id': gift_series.GiftId,
        'latitude': gift_series.Latitude,
        'longitude': gift_series.Longitude,
        'weight': gift_series.Weight
    }
    gift_list.append(gift)
    
end = time.time()
print("Time Taken to load dataset:", end-start)

## Generate initial population

In [ ]:
def generate_population(initial_population, i):
    if (i+1) % 100 == 0:
        print("generating population", i+1)
        
    random.shuffle(gift_list)
    total_weight = 0
    total_cost = 0
    trip_list = []

    j = 0
    trip_id = 1
    while j < len(gift_list):
        gift_trip_list = []
        total_weight = 0
        while j < len(gift_list) and (total_weight + gift_list[i]['weight']) <= SLEIGH_CAPACITY:
            curr_gift = gift_list[j]
            gift_trip_list.append(curr_gift)
            total_weight = total_weight + curr_gift['weight']
            j += 1

        trip_id += 1
        trip_cost = utils.tripCost(gift_trip_list)
        trip_order = {
            'gift_list': gift_trip_list,
            'trip_cost': trip_cost
        }
        total_cost += trip_cost
        trip_list.append(trip_order)
        
    redis_client.hset(f'ind-{i}', 'trip_list', json.dumps(trip_list))
    redis_client.hset(f'ind-{i}', 'total_cost', total_cost)

In [ ]:
start = time.time()
print("Initiating population...")

pool = multiprocessing.Pool(processes = multiprocessing.cpu_count()-1)
mp_initial_population = multiprocessing.Manager().list()
size_list = list(range(INITIAL_POPULATION_SIZE))
func = partial(generate_population, mp_initial_population)

pool.map(func, size_list)
pool.close()
pool.join()

end = time.time()
print("Time taken to init population:", end-start)

## Population generation

In [5]:
def sort_population_by_fitness():
    initial_population_index = []
    for i in range(INITIAL_POPULATION_SIZE):
        initial_population_index.append((i, float(redis_client.hget(f'ind-{i}', 'total_cost'))))
    sort_by_fitness = sorted(initial_population_index, key=lambda tup: tup[1])
    end = time.time()
    return sort_by_fitness

In [5]:
def mutate_individual(trip_list, total_cost):
    mutation_count = int(len(trip_list) * 0.01)
    for i in range(mutation_count):
        trip_index = random.randint(0, len(trip_list) - 1)
        chosen_trip = trip_list[trip_index]
        swapped_gift_list = utils.mutateGiftList(chosen_trip['gift_list'])
        temp_trip_cost = utils.tripCost(swapped_gift_list)

        if temp_trip_cost < chosen_trip['trip_cost']:
            total_cost = total_cost - chosen_trip['trip_cost'] + temp_trip_cost
            trip_list[trip_index]['gift_list'] = swapped_gift_list
            trip_list[trip_index]['trip_cost'] = temp_trip_cost
        
    return trip_list, total_cost

In [6]:
def crossover(trip_list, total_cost):
    crossover_count = int(len(trip_list) * 0.01)
    for i in range(crossover_count):
        trip_index_a, trip_index_b = utils.generateSwapIndices(len(trip_list))

        chosen_trip_a = trip_list[trip_index_a]
        chosen_trip_b = trip_list[trip_index_b]

        gift_list_a = chosen_trip_a['gift_list']
        gift_list_b = chosen_trip_b['gift_list']

        gift_list_a_total_len = len(gift_list_a)
        gift_list_b_total_len = len(gift_list_b)

        gift_list_a_half_len = int(gift_list_a_total_len / 2)
        gift_list_b_half_len = int(gift_list_b_total_len / 2)

        gift_list_a_half_1 = gift_list_a[:gift_list_a_half_len]
        gift_list_a_half_2 = gift_list_a[gift_list_a_half_len:]

        gift_list_b_half_1 = gift_list_b[:gift_list_b_half_len]
        gift_list_b_half_2 = gift_list_b[gift_list_b_half_len:]

        success = False
        while not success:
            new_gift_list_a = gift_list_a_half_1 + gift_list_b_half_2
            new_gift_list_b = gift_list_a_half_2 + gift_list_b_half_1

            new_gift_list_a_weight = utils.tripWeightUtil(new_gift_list_a, 0, len(new_gift_list_a)-1)
            new_gift_list_b_weight = utils.tripWeightUtil(new_gift_list_b, 0, len(new_gift_list_b)-1)

            if new_gift_list_a_weight <= SLEIGH_CAPACITY and new_gift_list_b_weight <= SLEIGH_CAPACITY:
                success = True
            else:
                gift_list_a_half_len += 1
                gift_list_b_half_len -= 1
                if gift_list_a_half_len >= gift_list_a_total_len or gift_list_b_half_len >= gift_list_b_total_len:
                    success = True
                else:
                    gift_list_a_half_1 = gift_list_a[:gift_list_a_half_len]
                    gift_list_a_half_2 = gift_list_a[gift_list_a_half_len:]
                    gift_list_b_half_1 = gift_list_b[:gift_list_b_half_len]
                    gift_list_b_half_2 = gift_list_b[gift_list_b_half_len:]

        # Check if new gene is better
        gift_list_a_cost = chosen_trip_a['trip_cost']
        gift_list_b_cost = chosen_trip_b['trip_cost']

        new_gift_list_a_cost = utils.tripCost(new_gift_list_a)
        new_gift_list_b_cost = utils.tripCost(new_gift_list_b)

        crossover_final_cost = (new_gift_list_a_cost - gift_list_a_cost) + (new_gift_list_b_cost - gift_list_b_cost)
        if crossover_final_cost < 0:
            trip_list[trip_index_a] = {
                'gift_list': new_gift_list_a,
                'trip_cost': new_gift_list_a_cost
            }
            trip_list[trip_index_b] = {
                'gift_list': new_gift_list_b,
                'trip_cost': new_gift_list_b_cost
            }

            total_cost += crossover_final_cost
        
    return trip_list, total_cost

In [13]:
def generate_new_population(iteration):
    sorted_population = sort_population_by_fitness()
    if iteration % 10000 == 0:
        print(f'iteration: {i} best_score: {sorted_population[0][1]}')
    
    mutation_method = random.randint(0, 1)
    if mutation_method == 0:
        picked_index = random.randint(INITIAL_POPULATION_SIZE - TOP_K, INITIAL_POPULATION_SIZE - 1) # LAST 10
    else:
        picked_index = random.randint(0, TOP_K - 1) # TOP 10
        
    chosen_index = sorted_population[picked_index][0]
    
    chosen_ind_trip_list = json.loads(redis_client.hget(f'ind-{chosen_index}', 'trip_list'))
    chosen_ind_total_cost = float(redis_client.hget(f'ind-{chosen_index}', 'total_cost'))
    
    chosen_ind_trip_list, chosen_ind_total_cost = mutate_individual(chosen_ind_trip_list, chosen_ind_total_cost)
    chosen_ind_trip_list, chosen_ind_total_cost = crossover(chosen_ind_trip_list, chosen_ind_total_cost)
    
    redis_client.hset(f'ind-{chosen_index}', 'trip_list', json.dumps(chosen_ind_trip_list))
    redis_client.hset(f'ind-{chosen_index}', 'total_cost', chosen_ind_total_cost)

In [ ]:
pool = multiprocessing.Pool(processes = multiprocessing.cpu_count()-1)
iterations = list(range(MAX_ITERATIONS))

pool.map(generate_new_population, iterations)
pool.close()
pool.join()

In [6]:
sorted_population = sort_population_by_fitness()

In [7]:
sorted_population[0][1]

382568826365.13855